## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

from netCDF4 import Dataset
from datetime import datetime, timedelta
from cftime import num2date, date2num

from datetime import datetime
import re

## Utils
To be further moved in dedicated api lib ...

In [ ]:
# Global attributes
with Dataset('output/tepco.nc') as rootgrp:
    print('Dataset global metadata')
    print('-' * 40)
    for name in rootgrp.ncattrs():
        print(f'{name}: {getattr(rootgrp, name)}')

Dataset global metadata
----------------------------------------
description: Template description
summary: Template summary
keyword: MARIS other-key-words
license: Common ...


In [ ]:
# Which groups & variables are present?
with Dataset('output/tepco.nc') as rootgrp:
    for grp in rootgrp.groups:
        print(f'Group (sample type): {grp}')
        print('# of measurements: ', rootgrp[f'/{grp}/sample'].shape[0]) 
        print('-' * 40)
        for name in rootgrp.groups[grp].variables:
            if name != 'sample':
                variable = rootgrp[f'/{grp}/{name}']
                print(variable.name)
                for attr_name in variable.ncattrs():
                    print(f'  - {attr_name}: {getattr(variable, attr_name)}')

Group (sample type): seawater
# of measurements:  21477
----------------------------------------
longitude
latitude
time
  - units: seconds since 1960-01-01 00:00:00.0
  - calendar: gregorian
i131
  - long_name: 131Iodine
  - standard_name: iodine_131
  - units: Bq/L
cs137
  - long_name: 137Cesium
  - standard_name: cesium_137
  - units: Bq/L
cs134
  - long_name: 134Cesium
  - standard_name: depth below sea level
  - units: m


In [ ]:
def nc_to_df(fname):
    pass

In [ ]:
with Dataset('output/tepco.nc') as ds:
    print(
    #print(ds.groups['seawater'])
    print('Cs137: ',  ds.groups['seawater'].variables['cs137'][:])
    print('longitude: ',  ds.groups['seawater'].variables['longitude'][:])
    #print(df.variables)

Cs137:  [5.3e+01 4.0e+01 5.5e+01 ... 1.0e-02     nan 1.5e-02]
longitude:  [141.03 141.03 141.03 ... 141.04 141.04 141.04]


In [ ]:
# Get dataframes
with Dataset('output/tepco.nc') as rootgrp:
    dfs = {}
    for grp in rootgrp.groups:
        series_dict = {}
        for name in rootgrp.groups[grp].variables:
            variable = rootgrp[f'/{grp}/{name}']
            if name == 'time':
                series_dict[name] = num2date(variable[:], 
                                             units='seconds since 1960-01-01 00:00:00.0',
                                             calendar = 'gregorian')
            else:
                series_dict[name] = variable[:]
        dfs[grp] = pd.DataFrame(series_dict)

In [ ]:
dfs['seawater']

,sample,longitude,latitude,time,i131,cs137,cs134
0,0,141.029999,37.32,2011-03-21 23:15:12,1100.0,53.000,48.0
1,1,141.029999,37.32,2011-03-22 14:28:16,1100.0,40.000,46.0
2,2,141.029999,37.32,2011-03-23 13:52:00,740.0,55.000,51.0
3,3,141.029999,37.32,2011-03-24 09:29:36,1100.0,94.000,99.0
4,4,141.029999,37.32,2011-03-25 09:59:28,430.0,34.000,26.0
...,...,...,...,...,...,...,...
21472,21472,141.039993,37.48,2022-07-12 10:10:08,NaN,0.011,NaN
21473,21473,141.039993,37.48,2022-07-12 10:10:08,NaN,NaN,NaN
21474,21474,141.039993,37.48,2022-07-19 09:59:28,NaN,0.010,NaN
21475,21475,141.039993,37.48,2022-07-19 09:59:28,NaN,NaN,NaN
